In [1]:
import os
import random
from io import open
import unicodedata
import string
import re

import torch
# import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pathlib import Path
import kaldi_io
import sys
import gc
import json
import time
from data_4 import AudioDataLoader, AudioDataset, pad_list

%matplotlib inline

print_use = False

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
train_json = "/home1/meichaoyang/workspace/git/espnet/egs/aishell2/asr1/dump/train_sp/deltafalse/data.json"
test_json = "/home1/meichaoyang/workspace/git/espnet/egs/aishell2/asr1/dump/test/deltafalse/data.json"
batch_size = 32
maxlen_in = 1000
maxlen_out = 50
num_workers = 4

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="3"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
tr_dataset = AudioDataset(train_json, batch_size,
                              maxlen_in, maxlen_out)
tr_loader = AudioDataLoader(tr_dataset, batch_size=1, num_workers=num_workers)

In [5]:
te_dataset = AudioDataset(test_json, batch_size,
                              maxlen_in, maxlen_out)
te_loader = AudioDataLoader(te_dataset, batch_size=1, num_workers=num_workers, shuffle=True)

In [6]:
char_list = []
char_list_path = "/home1/meichaoyang/workspace/git/espnet/egs/aishell2/asr3/data/lang_1char/train_units.txt"
with open(char_list_path, "r") as f:
    for line in f:
        data = line.split()
        char_list.append(data[0])

### 模型搭建

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, bidirectional=True, dropout=0.0):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.first = True
        self.rnn = nn.LSTM(input_size, hidden_size, 
                           batch_first=True,
                           dropout=dropout,
                           bidirectional=bidirectional)


    def forward(self, padded_input, input_lengths):
        
        total_length = padded_input.size(1)  # get the max sequence length
        packed_input = pack_padded_sequence(padded_input, input_lengths,
                                            batch_first=True)
        packed_output, hidden = self.rnn(packed_input)
        output, _ = pad_packed_sequence(packed_output,
                                        batch_first=True,
                                        total_length=total_length)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [30]:
class CTC_Model(nn.Module):
    
    def __init__(self, input_size, hidden_size, bidirectional=True, dropout=0.0):
        super(CTC_Model, self).__init__()
        self.encoder = Encoder(input_size, hidden_size,bidirectional, dropout)
        self.ctc_loss = nn.CTCLoss()
    
    def forward(self, padded_input, input_lengths, padded_target, target_lengths):
        
        encoder_output, _ = self.encoder(padded_input, input_lengths)
        encoder_output = torch.transpose(encoder_output,0,1)
        ctc_input = encoder_output.log_softmax(2)
        
        loss = self.ctc_loss(ctc_input, padded_target, input_lengths, target_lengths)
        
        return loss

### 训练

In [33]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [36]:
def trainIters(model, epoch, optimizier, print_every=10, plot_every=10, learning_rate=0.01):

    start = time.time()
    n_iters = len(tr_dataset)
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    
    criterion = nn.NLLLoss()

    for e in range(epoch):
        for i, (data) in enumerate(tr_loader):
            padded_input, input_lengths, padded_target, target_lengths= data
            padded_input = padded_input.cuda()
            input_lengths = input_lengths.cuda()
            padded_target = padded_target.cuda()
            target_lengths = target_lengths.cuda()
            loss = model(padded_input, input_lengths, padded_target, target_lengths)
    #         print(loss) #.requires_grad
            print_loss_total += float(loss)
            plot_loss_total += float(loss)

            optimizier.zero_grad()
            loss.backward()

            optimizier.step()

            if (i+1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                txt = 'Epoch %d | Iter %d | %s (%d %d%%) %.4f' % (e+1, i+1, timeSince(start, (e *n_iters +i+1) / (n_iters*epoch)),
                                             (i+1), (e *n_iters +i+1) / (n_iters*epoch) * 100, print_loss_avg)
                print(txt)

            if i+1 % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0


In [39]:
input_size = 83

hidden_size = 256
vocab_size = len(char_list)
embedding_dim = 512
sos_id = 0
eos_id = 1
learning_rate = 1e-3
momentum = 0
l2 = 1e-5

IGNORE_ID=-1


model = CTC_Model(input_size, hidden_size)
print(model)
model.cuda()

optimizier = torch.optim.Adam(model.parameters(),
                                     lr=learning_rate,
#                                      momentum=momentum,
                                     weight_decay=l2)
trainIters(model, 1,optimizier, print_every=100)

CTC_Model(
  (encoder): Encoder(
    (rnn): LSTM(83, 256, batch_first=True, bidirectional=True)
  )
  (ctc_loss): CTCLoss()
)
Epoch 1 | Iter 100 | 0m 12s (- 183m 53s) (100 0%) 275.1038
Epoch 1 | Iter 200 | 0m 22s (- 163m 38s) (200 0%) 224.6259
Epoch 1 | Iter 300 | 0m 35s (- 170m 25s) (300 0%) 213.2085


RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED